In [31]:
import pandas as pd

# Read in the data from nfl_scores_raw.csv
nfl_scores = pd.read_csv('nfl_scores_raw.csv')

# drop all rows with 'schedule_season' < 1979
nfl_scores = nfl_scores[nfl_scores['schedule_season'] >= 1979]

nfl_scores.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
2501,09.01.79,1979,1,False,Tampa Bay Buccaneers,31,16,Detroit Lions,TB,-3.0,30,Houlihan's Stadium,False,79.0,9.0,87.0,NaN
2502,09.02.79,1979,1,False,Buffalo Bills,7,9,Miami Dolphins,MIA,-5.0,39,Ralph Wilson Stadium,False,74.0,15.0,74.0,NaN
2503,09.02.79,1979,1,False,Chicago Bears,6,3,Green Bay Packers,CHI,-3.0,31,Soldier Field,False,78.0,11.0,68.0,NaN
2504,09.02.79,1979,1,False,Denver Broncos,10,0,Cincinnati Bengals,DEN,-3.0,31.5,Mile High Stadium,False,69.0,6.0,38.0,NaN
2505,09.02.79,1979,1,False,Kansas City Chiefs,14,0,Baltimore Colts,KC,-1.0,37,Arrowhead Stadium,False,76.0,8.0,71.0,NaN


In [32]:
# clean the team names by replacing old team names with new team names
team_name_cleanup = {'St. Louis Rams': 'Los Angeles Rams', 
                     'St. Louis Cardinals': 'Arizona Cardinals',
                     'Washington Redskins': 'Washington Commanders',
                     'Washington Football Team': 'Washington Commanders',
                     'Baltimore Colts': 'Indianapolis Colts',
                     'San Diego Chargers': 'Los Angeles Chargers',
                     'Houston Oilers': 'Tennessee Titans',
                     'Oakland Raiders': 'Las Vegas Raiders',
                     'Los Angeles Raiders': 'Las Vegas Raiders',
                     'Phoenix Cardinals': 'Arizona Cardinals',
                     'Tennessee Oilers': 'Tennessee Titans',}

nfl_scores['team_home'] = nfl_scores['team_home'].replace(team_name_cleanup)
nfl_scores['team_away'] = nfl_scores['team_away'].replace(team_name_cleanup)

In [33]:
teams = nfl_scores['team_home'].unique()

# create a column 'winner' which is the team that won the game ('tie' if the game was a tie)
nfl_scores['winner'] = nfl_scores.apply(lambda row: 'tie' if row['score_home'] == row['score_away'] else row['team_home'] if row['score_home'] > row['score_away'] else row['team_away'], axis=1)

nfl_scores.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,winner
2501,09.01.79,1979,1,False,Tampa Bay Buccaneers,31,16,Detroit Lions,TB,-3.0,30,Houlihan's Stadium,False,79.0,9.0,87.0,NaN,Tampa Bay Buccaneers
2502,09.02.79,1979,1,False,Buffalo Bills,7,9,Miami Dolphins,MIA,-5.0,39,Ralph Wilson Stadium,False,74.0,15.0,74.0,NaN,Miami Dolphins
2503,09.02.79,1979,1,False,Chicago Bears,6,3,Green Bay Packers,CHI,-3.0,31,Soldier Field,False,78.0,11.0,68.0,NaN,Chicago Bears
2504,09.02.79,1979,1,False,Denver Broncos,10,0,Cincinnati Bengals,DEN,-3.0,31.5,Mile High Stadium,False,69.0,6.0,38.0,NaN,Denver Broncos
2505,09.02.79,1979,1,False,Kansas City Chiefs,14,0,Indianapolis Colts,KC,-1.0,37,Arrowhead Stadium,False,76.0,8.0,71.0,NaN,Kansas City Chiefs


In [34]:
# create a list of all the columns that will be created
columns = []
for i in range(1, 11):
    columns.append('home_' + str(i))
    columns.append('away_' + str(i))

# append the new columns to the dataframe
for column in columns:
    nfl_scores[column] = ''

# create a list of all the teams
teams = nfl_scores['team_home'].unique()

# create a list of all the seasons
seasons = nfl_scores['schedule_season'].unique()

# create a dict with the max week for each season (only numeric weeks are included)
season_max_week = {}
for season in seasons:
    season_max_week[season] = nfl_scores[(nfl_scores['schedule_season'] == season) & (nfl_scores['schedule_week'].str.isnumeric())]['schedule_week'].astype(int).max()

print(season_max_week)

{1979: 16, 1980: 16, 1981: 16, 1982: 9, 1983: 16, 1984: 16, 1985: 16, 1986: 16, 1987: 12, 1988: 16, 1989: 16, 1990: 17, 1991: 17, 1992: 17, 1993: 18, 1994: 17, 1995: 17, 1996: 17, 1997: 17, 1998: 17, 1999: 17, 2000: 17, 2001: 17, 2002: 17, 2003: 17, 2004: 17, 2005: 17, 2006: 17, 2007: 17, 2008: 17, 2009: 17, 2010: 17, 2011: 17, 2012: 17, 2013: 17, 2014: 17, 2015: 17, 2016: 17, 2017: 17, 2018: 17, 2019: 17, 2020: 17, 2021: 18, 2022: 18, 2023: 9}


In [35]:
# for each team, create a dict with the season and week as key and the result of the game (win/loss/tie) as value
teams_results = {}
for team in teams:
    team_df = nfl_scores[(nfl_scores['team_home'] == team) | (nfl_scores['team_away'] == team)]
    teams_results[team] = {}
    for season in seasons:
        team_season_df = team_df[team_df['schedule_season'] == season]

        # if team didn't play in that season, skip
        if len(team_season_df) == 0:
            continue

        # create a list with win/loss/tie for each game
        results = []
        for index, row in team_season_df.iterrows():
            if row['winner'] == 'tie':
                results.append('tie')
            elif row['winner'] == team:
                results.append('win')
            else:
                results.append('loss')
        
        teams_results[team][season] = results


In [36]:
# for each row starting from season 1979 week 11, create columns with the results of the last 10 games for each team
# (e.g. 'home_10' indicates 'win'/'loss'/'tie' for the home team 10 weeks ago, 'away_1' indicates 'win'/'loss'/'tie' for the away team last week)
# if a team has played less than 10 games, fill the remaining columns with 'N/A'

for index, row in nfl_scores.iterrows():
    home_team = row['team_home']
    away_team = row['team_away']
    season = row['schedule_season']
    week = row['schedule_week']
    if week.isnumeric():
        week = int(week)
    else:
        week = season_max_week[season]
    
    # create a list with the results of the last 10 games for the home team
    home_results = []
    week_prev_season = -1
    week_two_seasons_ago = -1
    for i in range(1, 11):
        if week - i - 1 < 0:
            if season - 1 in teams_results[home_team].keys():
                if abs(week_prev_season) > len(teams_results[home_team][season - 1]):
                    home_results.append(teams_results[home_team][season - 2][week_two_seasons_ago]) # 1982 only had 9 games so we need to go back 2 seasons
                    week_two_seasons_ago -= 1
                else:
                    home_results.append(teams_results[home_team][season - 1][week_prev_season])
                    week_prev_season -= 1
            else:
                home_results.append('N/A')
        else:
            while week - i - 1 >= len(teams_results[home_team][season]):
                week -= 1
            home_results.append(teams_results[home_team][season][week - i - 1])

    # create a list with the results of the last 10 games for the away team
    away_results = []
    week_prev_season = -1
    week_two_seasons_ago = -1
    for i in range(1, 11):
        if week - i - 1 < 0:
            if season - 1 in teams_results[away_team].keys():
                if abs(week_prev_season) > len(teams_results[away_team][season - 1]):
                    away_results.append(teams_results[away_team][season - 2][week_two_seasons_ago]) # 1982 only had 9 games so we need to go back 2 seasons
                    week_two_seasons_ago -= 1
                else:
                    away_results.append(teams_results[away_team][season - 1][week_prev_season])
                    week_prev_season -= 1
            else:
                away_results.append('N/A')
        else:
            while week - i - 1 >= len(teams_results[away_team][season]):
                week -= 1
            away_results.append(teams_results[away_team][season][week - i - 1])
    
    # fill the columns with the results
    for i in range(1, 11):
        nfl_scores.at[index, 'home_' + str(i)] = home_results[i - 1]
        nfl_scores.at[index, 'away_' + str(i)] = away_results[i - 1]


In [37]:
# create a dict with all team abbreviations
team_abbreviations = {'Arizona Cardinals': 'ARI',
                      'Atlanta Falcons': 'ATL',
                        'Baltimore Ravens': 'BAL',
                        'Buffalo Bills': 'BUF',
                        'Carolina Panthers': 'CAR',
                        'Chicago Bears': 'CHI',
                        'Cincinnati Bengals': 'CIN',
                        'Cleveland Browns': 'CLE',
                        'Dallas Cowboys': 'DAL',
                        'Denver Broncos': 'DEN',
                        'Detroit Lions': 'DET',
                        'Green Bay Packers': 'GB',
                        'Houston Texans': 'HOU',
                        'Indianapolis Colts': 'IND',
                        'Jacksonville Jaguars': 'JAX',
                        'Kansas City Chiefs': 'KC',
                        'Las Vegas Raiders': 'LVR',
                        'Los Angeles Chargers': 'LAC',
                        'Los Angeles Rams': 'LAR',
                        'Miami Dolphins': 'MIA',
                        'Minnesota Vikings': 'MIN',
                        'New England Patriots': 'NE',
                        'New Orleans Saints': 'NO',
                        'New York Giants': 'NYG',
                        'New York Jets': 'NYJ',
                        'Philadelphia Eagles': 'PHI',
                        'Pittsburgh Steelers': 'PIT',
                        'San Francisco 49ers': 'SF',
                        'Seattle Seahawks': 'SEA',
                        'Tampa Bay Buccaneers': 'TB',
                        'Tennessee Titans': 'TEN',
                        'Washington Commanders': 'WAS',
}

# invert the dict
team_abbreviations = {v: k for k, v in team_abbreviations.items()}

# replace the abbreviations with the full team names in the 'team_favorite_id' column
nfl_scores['team_favorite_id'] = nfl_scores['team_favorite_id'].replace(team_abbreviations)

# in the 'team_favorite_id' column, replace the team names with 'home' or 'away' depending on which team is the favorite
nfl_scores['team_favorite_id'] = nfl_scores.apply(lambda row: 'home' if row['team_home'] == row['team_favorite_id'] else 'away' if row['team_away'] == row['team_favorite_id'] else 'PICK', axis=1)

In [38]:
# replace the 'winner' column with 'home' or 'away' depending on which team won the game
nfl_scores['winner'] = nfl_scores.apply(lambda row: 'home' if row['team_home'] == row['winner'] else 'away' if row['team_away'] == row['winner'] else 'tie', axis=1)

In [39]:
# drop unnecessary columns
columns_to_drop = ['stadium', 'stadium_neutral', 'score_home', 'score_away']
nfl_scores = nfl_scores.drop(columns=columns_to_drop)

# save the dataframe to a csv file
nfl_scores.to_csv('nfl_scores_processed.csv', index=False)